# 3.2 Operators based on second derivative

There are more advanced methods for edge detection than first-derivative based.  This notebook covers some second-derivative based methods ike:

- LaPlace operator
- LoG operator
- Canny algorithm

We will not implement those operators because they are more complex and it may take some time. Luckily, they are implemented in openCV (`cv2.Laplacian` and  `cv2.Canny`). 

## Problem context - Edge detection for medical images

Unfortunately, you were not accepted by researching team at Hospital Clínico because results were not as good as you expected. Anyway, they show you the algorithms that they are currently using so you can study it for next interview.

<img src="./images/hired.jpg" width="400">

In [5]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib
from ipywidgets import interact, fixed, widgets
matplotlib.rcParams['figure.figsize'] = (15.0, 15.0)

images_path = './images/'

### Laplace and LoG operators 



Compared with the first derivative-based edge detectors such as Sobel operator, the Laplacian operator may yield better results in edge localization.  

#### Implementation

A first derivative is used (openCV uses Sobel but any is valid): $\\[5pt]$

$$\frac{\partial f(x,y)}{\partial x} = f(i+1,j) - f(i,j) \\[5pt]$$

$$\frac{\partial f(x,y)}{\partial y} = f(i,j+1) - f(i,j) \\[5pt]$$

Then, take second derivatives:

$$g = \frac{\partial f^2}{\partial x^2} = f(i+1,j) - 2f(i,j) + f(i-1,j) \\[5pt]$$

$$h = \frac{\partial f^2}{\partial y^2} = f(i,j+1) - 2f(i,j) + f(i,j-1) \\[10pt]$$

Finally, apply distributive property of convolution:

<img src="./images/laplace.png" width="300">

Unfortunately, the Laplacian operator is very sensitive to noise. If image is blurred using a Gaussian filter before applying Laplace operator, it is called **LoG** (Laplace of Gaussian).


**What to do?** First, complete the function **gaussian_smoothing** that blur an image using a Gaussian filter. Then, normalizes and returns resulting image.

In [6]:
def gaussian_smoothing(image, sigma, w_kernel):
    """ Blur and normalize input image.   
    
        Args:
            image: Input image to be binarized
            sigma: Standard deviation of the Gaussian distribution
            w_kernel: Kernel aperture size
                    
        Returns: 
            binarized: Blurred image
    """   
    
    # Define 1D kernel
    s=sigma
    w=w_kernel
    kernel_1D = [np.exp(-z*z/(2*s*s))/np.sqrt(2*np.pi*s*s) for z in range(-w,w+1)]
    
    # Apply distributive property of convolution
    kernel_2D = np.outer(kernel_1D,kernel_1D)
    
    # Blur image
    smoothed_img = cv2.filter2D(image,cv2.CV_8U,kernel_2D)
    
    # Normalize to [0 254] values
    smoothed_norm = np.array(image.shape)
    smoothed_norm = cv2.normalize(smoothed_img,None, 0, 255, cv2.NORM_MINMAX)
    
    return smoothed_norm

Now, we are going to see the differences between Laplace operator and LoG. 

**Your job is to** complete **laplace_testing** that applies Laplacian operator to an image and to the previously blurred image. Then both are displayed along the original image. It takes an image, the size of the Laplacian filter (odd), and parameters of the gaussian filter as input.

Note that it is possible to reduce computation time if LoG is precomputed. This is convolving Laplace and Gaussian filters instead of applying them separatelly.

*Tip: openCV defines Laplace operator as [cv2.Laplacian()](https://docs.opencv.org/3.4/d5/db5/tutorial_laplace_operator.html)*

In [7]:
def laplace_testing(image, size_Laplacian, sigma, w_gaussian):
    """ Apply Laplacian and Log operators to an image.   
    
        Args:
            image: Input image to be binarized
            size_Laplacian: size of Laplacian kernel (odd)
            sigma: Standard deviation of the Gaussian distribution
            w_gaussian: Gaussian kernel aperture size
    """  
    
    # Blur image
    blurred_img = gaussian_smoothing(image, sigma, w_gaussian)
    
    # Apply Laplacian to original image
    laplacian = cv2.Laplacian(image, cv2.CV_16S, ksize=size_Laplacian)
    
    # Aplay Laplacian to blurred image
    laplacian_blurred = cv2.Laplacian(blurred_img, cv2.CV_16S, ksize=size_Laplacian)
    
    # Show initial image
    plt.subplot(131)
    plt.imshow(image, cmap='gray')
    plt.title('Original image')
    
    # Show laplacian
    plt.subplot(132)
    plt.imshow(laplacian, cmap='gray')
    plt.title('Laplacian without blurring')
    
    # Show LoG
    plt.subplot(133)
    plt.imshow(laplacian_blurred, cmap='gray')
    plt.title('Laplacian blurred (LoG)')

**What to do?** Try this method and play with interactive parameters.$\\[5pt]$      
After that, **answer following questions**:

- Can be Laplacian applied without a previous blurring?
- What would be needed for obtaining the edges from those images?

In [8]:
image = cv2.imread(images_path + 'medical_3.jpg', 0)
interact(laplace_testing, image=fixed(image), size_Laplacian=(1,7,2), sigma=(1,3,0.1), w_gaussian=(1,3,1))

interactive(children=(IntSlider(value=3, description='size_Laplacian', max=7, min=1, step=2), FloatSlider(valu…

<function __main__.laplace_testing(image, size_Laplacian, sigma, w_gaussian)>

### Canny algorithm

Canny edge detector is an algorithm that applies DroG operator, non-maxima suppression and hysteresis for edge detection in images.

Steps:

1. Filter out any noise. The Gaussian filter is used for this purpose.$\\[5pt]$

2. Apply DroG.$\\[5pt]$

3. *Non-maximum suppression* is applied. This removes pixels that are not considered to be part of an edge. Hence, only thin lines (candidate edges) will remain. For this, get the maximum of the neighbors pixels in the gradient direction.$\\[5pt]$

<img src="./images/canny_nonmaxima.png" width="800">

4. *Hysteresis*: The final step. Canny does use two thresholds (upper and lower):

   - If a pixel gradient is higher than the upper threshold, the pixel is accepted as an edge. 
   - If a pixel gradient value is below the lower threshold, then it is rejected. 
   - If the pixel gradient is between the two thresholds, then it will be accepted only if it is connected to a pixel that is above the upper threshold.
   
<img src="./images/hysteresis.png" width="800">
   
**What to do?** Complete **canny_testing**, that applies Canny algorithm to an image and to that image but previously blurred *(note that openCV Canny's implementation does not apply Gaussian blurring)*. Then both are displayed along the original image. It takes an image, both lower and upper Canny thresholds, and parameters of the gaussian filter as input.

*Tip: openCV defines canny algorithm as [cv2.Canny()](https://docs.opencv.org/2.4/modules/imgproc/doc/feature_detection.html?highlight=canny)*

In [9]:
def canny_testing(image, lower_threshold, upper_threshold, sigma, w_gaussian):
    """ Apply Canny algorithm to an image.   
    
        Args:
            image: Input image to be binarized
            lower_threshold: bottom value for hysteresis
            upper_threshold: top value for hysteresis
            sigma: Standard deviation of the Gaussian distribution
            w_gaussian: Gaussian kernel aperture size
    """  
    
    # Blur image
    blurred_img = gaussian_smoothing(image,sigma,w_gaussian)
    
    # Apply Canny to original image
    canny = cv2.Canny(image,lower_threshold,upper_threshold)
    
    # Apply Canny to blurred image
    canny_blurred = cv2.Canny(blurred_img,lower_threshold,upper_threshold)

    # Show initial image
    plt.subplot(131)
    plt.imshow(image, cmap='gray')
    plt.title('Original image')
    
    # Show Canny without blurring
    plt.subplot(132)
    plt.imshow(canny, cmap='gray')
    plt.title('Canny without blurring')
    
    # Show Canny with blurring
    plt.subplot(133)
    plt.imshow(canny_blurred, cmap='gray')
    plt.title('Canny blurred')

**What to do?** Try this method and play with interactive parameters.$\\[5pt]$      
After that, **answer following questions**:
- Can be Canny applied without a previous blurring?
- What is a *good* value for both thresholds

In [10]:
image = cv2.imread(images_path + 'medical_2.jpg', 0)
interact(canny_testing, image=fixed(image), lower_threshold=(0,260,20), upper_threshold=(0,260,20), sigma=(1,3,0.1), w_gaussian=(1,3,1))

interactive(children=(IntSlider(value=120, description='lower_threshold', max=260, step=20), IntSlider(value=1…

<function __main__.canny_testing(image, lower_threshold, upper_threshold, sigma, w_gaussian)>

## Conclusion

Terrific! You finished this notebook, that includes information about

- Laplace operator and the importance of smoothing.
- how to use canny alorithm, and how it is implemented.

**Extra**

Canny algorithm is a very known algorithm in the field, it is used in a lot of current technologies. Although it gets very good results, original paper was published in 1986 by John Canny. 

If you have curiosity about the original paper, you can download it [here](https://www.google.com/url?sa=t&rct=j&q=&esrc=s&source=web&cd=10&ved=2ahUKEwiU9uyiganoAhWNDWMBHducCvsQFjAJegQIBhAB&url=http%3A%2F%2Fciteseerx.ist.psu.edu%2Fviewdoc%2Fdownload%3Fdoi%3D10.1.1.420.3300%26rep%3Drep1%26type%3Dpdf&usg=AOvVaw3tsKoxnc3qnS7bji3HmvQc).